In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import pandas as pd
import time
from PIL import Image
from pytesseract import pytesseract
from io import BytesIO
import base64
import pytesseract
import warnings
from selenium import webdriver
import time
import chromedriver_autoinstaller as chromedriver


In [ ]:
chromedriver.install()

In [53]:
warnings.filterwarnings('ignore')

In [67]:
t0 = time.time()
global carsData
driver = webdriver.Chrome('C:\\Users\\saad\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\chromedriver_autoinstaller\\105\\chromedriver.exe')
base_url = "https://syarah.com/en/cars/used-cars?sort=-record_posted_date"
delay = 3
driver.get(base_url)
#driver.find_element_by_link_text("All").click()
for i in range(1,120):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
html_source = driver.page_source
soup = BeautifulSoup(html_source,'html.parser')
soup = soup.find('div',{'class':'allCarsResult'})
carsData = soup
t1 = time.time()
print('\n\nTime comsumed: ', t1-t0)
print('done');



Time comsumed:  507.3195917606354
done


In [45]:
items=carsData.findAll('a',{'class':'SearchCard-module__card'})
for item in items[0:3]:
    print(item['href'])
    item=item
    name= item.find('strong',{'class':'SearchCard-module__title'}).find(text=True)
    price= item.find('div',{'class':'SearchCard-module__beforeDiscount'}).find(text=True)
    print(name,price)

/en/car/Kia-Sportage-2019-Used-140996?sol=1
Kia Sportage  2019 Standard Double 69,000
/en/car/Toyota-Yaris-2021-Used-140993?sol=1
Toyota Yaris   2021 Standard  51,000
/en/car/Toyota-Yaris-2021-Used-140991?sol=1
Toyota Yaris   2021 Standard  51,000


In [68]:
def getDetailsData(detailsUrl):
    #print(detailsUrl)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    detialRequest = requests.get(detailsUrl,headers=headers)
    detailContent = detialRequest.content
    detailSoup = BeautifulSoup(detailContent,features="html.parser")
    details_container=detailSoup.find('section',{'class':'carInfoCC'})
    #location=details_container.find('div',{'class','location'})
    return details_container


In [69]:
global df
df=pd.DataFrame()

In [79]:
def getCarsData():
    global df
    #data=[]
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}

    items=carsData.findAll('a',{'class':'SearchCard-module__card'})
    print(len(items))
    for index,item in enumerate(items[500:]):
        try:
            features={}
            title=item.find('strong',{'class':'SearchCard-module__title'}).find(text=True)
            price=item.find('div',{'class':'SearchCard-module__beforeDiscount'}).find(text=True)
            #get more data from details page
            detailsUrl='https://syarah.com'+item['href']
            print(index,detailsUrl)
            details_html=getDetailsData(detailsUrl)
            carInfo=details_html.find('div',{'class','carDetailesCC'}).findAll('div')
            #print(carInfo)
            for info in carInfo:
                col_name=info.find('label').find(text=True).lower()
                value=info.find('strong').find(text=True)
                #print(col_name,value)
                features[col_name]=value

    #         brand=carInfo[0].find('strong').find(text=True)
    #         model=carInfo[1].find('strong').find(text=True)
    #         trim=carInfo[2].find('strong').find(text=True)
    #         model_year=carInfo[3].find('strong').find(text=True)
    #         specs=carInfo[4].find('strong').find(text=True) #origin of car
    #         color=carInfo[5].find('strong').find(text=True) 
    #         options=carInfo[6].find('strong').find(text=True) 
    #         fuel_type=carInfo[7].find('strong').find(text=True) 
    #         transmission_type=carInfo[8].find('strong').find(text=True) 
    #         drive_type=carInfo[9].find('strong').find(text=True) 
    #         number_of_cylinders=carInfo[10].find('strong').find(text=True) 
    #         engine_size=carInfo[11].find('strong').find(text=True) 
    #         odo=carInfo[13].find('strong').find(text=True) 
            location='Riyadh' #because it is a company based not individual user

            aminities_html=details_html.findAll('div',{'class','singleinfoRowN'})
    #         print(aminities_html)
            #features={}
            aminities=[]
            #print(features_html[:-1])
            for category in aminities_html:
                li_items=category.findAll('li')
                for index,li_item in enumerate(li_items):
                    aminity=li_item.find('span').find(text=True)
                    aminities.append(aminity)
    #         features['title']=title
    #         features['brand']=brand
    #         features['model']=model
    #         features['trim']=trim

    #         features['price']=price
    #         features['model_year']=model_year
    #         features['odo']=odo
    #         features['aminities']=aminities

    #         features['specs']=specs
    #         features['number_of_seats']=0
    #         features['drive_type']=drive_type
    #         features['number_of_cylinders']=number_of_cylinders
    #         features['color']=color 
    #         features['fuel_type']=fuel_type
    #         features['options']=options

    #         features['transmission_type']=transmission_type
    #         features['drive_type']=drive_type
    #         features['engine_size']=engine_size

    #         features['number_of_cylinders']=number_of_cylinders
            features['location']=location
            features['aminities']=aminities

            df=df.append(features,ignore_index=True)
            #print(index,title,'   |',price,'  |',model_year,'  |',odo,' |',location," |",aminities)
            #break
        except Exception as ex :
            print(ex)
            continue


In [80]:
t0 = time.time()
getCarsData()
t1 = time.time()
print('\n\nTime comsumed: ', t1-t0)
#df.to_csv('../Data/car_switch_data_2.csv',index=False)

1186
0 https://syarah.com/en/car/BMW-X_Series-2021-Used-137454?sol=1
1 https://syarah.com/en/car/Chevrolet-Spark-2020-Used-137450?sol=1
2 https://syarah.com/en/car/Hyundai-Elantra-2020-Used-137447?sol=1
3 https://syarah.com/en/car/Kia-Pegas-2020-Used-137389?sol=1
4 https://syarah.com/en/car/SsangYong-Tivoli-2021-Used-137327?sol=1
5 https://syarah.com/en/car/Hyundai-Accent-2020-Used-137308?sol=1
6 https://syarah.com/en/car/Hyundai-Accent-2020-Used-137304?sol=1
7 https://syarah.com/en/car/Hyundai-Elantra-2020-Used-137275?sol=1
8 https://syarah.com/en/car/Chevrolet-Spark-2020-Used-137269?sol=1
9 https://syarah.com/en/car/GMC-Yukon-SLE-2019-Used-137268?sol=1
10 https://syarah.com/en/car/Ford-Explorer-2021-Used-137253?sol=1
11 https://syarah.com/en/car/Audi-A6-2019-Used-137250?sol=1
12 https://syarah.com/en/car/Hyundai-Accent-2019-Used-137249?sol=1
13 https://syarah.com/en/car/Toyota-Yaris-2019-Used-137248?sol=1
14 https://syarah.com/en/car/Hyundai-Accent-2019-Used-137247?sol=1
15 https://s

123 https://syarah.com/en/car/Nissan-Patrol-TITANIUM-2022-Used-135575?sol=1
124 https://syarah.com/en/car/Hyundai-Accent-2019-Used-135564?sol=1
125 https://syarah.com/en/car/Toyota-Avalon-Limited-2022-Used-135563?sol=1
126 https://syarah.com/en/car/Hyundai-Accent-2019-Used-135513?sol=1
127 https://syarah.com/en/car/Hyundai-Accent-2019-Used-135509?sol=1
128 https://syarah.com/en/car/Changan-CS_75-2021-Used-135504?sol=1
129 https://syarah.com/en/car/BMW-4_Series-2017-Used-135500?sol=1
130 https://syarah.com/en/car/Toyota-Sequoia-2016-Used-135459?sol=1
131 https://syarah.com/en/car/Chevrolet-Tahoe-2020-Used-135456?sol=1
132 https://syarah.com/en/car/Hyundai-Accent-2019-Used-135453?sol=1
133 https://syarah.com/en/car/Hyundai-Santafe-2016-Used-135447?sol=1
134 https://syarah.com/en/car/Ford-Taurus-2023-Used-135378?sol=1
135 https://syarah.com/en/car/Toyota-Hilux-GLX-2019-Used-135376?sol=1
136 https://syarah.com/en/car/BMW-X_Series-2022-Used-135375?sol=1
137 https://syarah.com/en/car/Hyundai

244 https://syarah.com/en/car/Hyundai-Elantra-2020-Used-133549?sol=1
245 https://syarah.com/en/car/Hyundai-Accent-2020-Used-133497?sol=1
246 https://syarah.com/en/car/Hyundai-Elantra-2020-Used-133495?sol=1
247 https://syarah.com/en/car/Toyota-Fortuner-2015-Used-133490?sol=1
248 https://syarah.com/en/car/MG-5-2021-Used-133488?sol=1
249 https://syarah.com/en/car/Nissan-Sunny-2022-Used-133487?sol=1
250 https://syarah.com/en/car/Toyota-Camry-GLE-2020-Used-133484?sol=1
251 https://syarah.com/en/car/Cadillac-CT5-2022-Used-133426?sol=1
252 https://syarah.com/en/car/Hyundai-Santafe-2022-Used-133423?sol=1
253 https://syarah.com/en/car/HAVAL-Jolion-2022-Used-133422?sol=1
254 https://syarah.com/en/car/Toyota-Land_Cruiser-2022-Used-133421?sol=1
255 https://syarah.com/en/car/GMC-Yukon-SLE-2020-Used-133406?sol=1
256 https://syarah.com/en/car/Toyota-FJ_Cruiser-2022-Used-133309?sol=1
257 https://syarah.com/en/car/Changan-CS85-2022-Used-133304?sol=1
258 https://syarah.com/en/car/Hyundai-Sonata-2018-Use

365 https://syarah.com/en/car/Toyota-RAV4-2021-Used-130885?sol=1
366 https://syarah.com/en/car/Ford-Taurus-SE-2019-Used-130870?sol=1
367 https://syarah.com/en/car/Toyota-RAV4-2018-Used-130855?sol=1
'NoneType' object has no attribute 'find'
368 https://syarah.com/en/car/Toyota-Corolla-XLI-2019-Used-130854?sol=1
369 https://syarah.com/en/car/Hyundai-Sonata-2019-Used-130844?sol=1
370 https://syarah.com/en/car/Hyundai-Elantra-2020-Used-130816?sol=1
371 https://syarah.com/en/car/Isuzu-D-max-2019-Used-130814?sol=1
372 https://syarah.com/en/car/Toyota-Hilux-SGLX-2017-Used-130811?sol=1
373 https://syarah.com/en/car/Hyundai-Accent-2019-Used-130807?sol=1
374 https://syarah.com/en/car/Jeep-Compass-2018-Used-130806?sol=1
375 https://syarah.com/en/car/Toyota-Camry-2019-Used-130802?sol=1
376 https://syarah.com/en/car/Mercedes-C-2021-Used-130796?sol=1
377 https://syarah.com/en/car/Toyota-Camry-2020-Used-130784?sol=1
378 https://syarah.com/en/car/Kia-Sportage-2018-Used-130773?sol=1
379 https://syarah.

486 https://syarah.com/en/car/Mercedes-S-2018-Used-127702?sol=1
487 https://syarah.com/en/car/Ford-Mustang-2018-Used-127662?sol=1
488 https://syarah.com/en/car/Changan-CS95-2022-Used-127661?sol=1
489 https://syarah.com/en/car/Hyundai-Accent-2019-Used-127594?sol=1
490 https://syarah.com/en/car/Hyundai-Accent-2019-Used-127593?sol=1
491 https://syarah.com/en/car/Hyundai-Tucson-2022-Used-127589?sol=1
492 https://syarah.com/en/car/Honda-Accord-2018-Used-127573?sol=1
493 https://syarah.com/en/car/Toyota-Avalon-Limited-2019-Used-127571?sol=1
494 https://syarah.com/en/car/Toyota-Camry-GLE-2019-Used-127570?sol=1
495 https://syarah.com/en/car/Mini-Countryman-2018-Used-127568?sol=1
496 https://syarah.com/en/car/BMW-7_Series-2018-Used-127567?sol=1
497 https://syarah.com/en/car/Jeep-Wrangler-2018-Used-127562?sol=1
498 https://syarah.com/en/car/Hyundai-Azera-2018-Used-127556?sol=1
499 https://syarah.com/en/car/Jeep-Wrangler-2019-Used-127553?sol=1
500 https://syarah.com/en/car/Toyota-RAV4-2020-Used-1

607 https://syarah.com/en/car/Chrysler-300-2019-Used-122076?sol=1
608 https://syarah.com/en/car/Kia-K5-2021-Used-122041?sol=1
609 https://syarah.com/en/car/Ford-F150-Raptor-2020-Used-121947?sol=1
610 https://syarah.com/en/car/Mitsubishi-Pajero-2017-Used-121865?sol=1
611 https://syarah.com/en/car/Kia-Cerato-2020-Used-121579?sol=1
612 https://syarah.com/en/car/Toyota-Camry-SE-2019-Used-121155?sol=1
613 https://syarah.com/en/car/Lexus-NX-2021-Used-121151?sol=1
614 https://syarah.com/en/car/MG-RX5-2018-Used-121146?sol=1
615 https://syarah.com/en/car/Kia-Sportage-2019-Used-121132?sol=1
616 https://syarah.com/en/car/Toyota-Hilux-2018-Used-121109?sol=1
617 https://syarah.com/en/car/Toyota-Hilux-2017-Used-121085?sol=1
618 https://syarah.com/en/car/Kia-K5-2021-Used-121084?sol=1
619 https://syarah.com/en/car/Toyota-RAV4-2018-Used-120984?sol=1
620 https://syarah.com/en/car/Toyota-RAV4-2018-Used-120983?sol=1
621 https://syarah.com/en/car/Hyundai-Sonata-2019-Used-120936?sol=1
622 https://syarah.com

In [83]:
df.to_csv('../Data/Syarah_data.csv',index=False)

In [84]:
df_all=pd.read_csv('../Data/Syarah_data.csv')

In [88]:
df_all.head()


,brand,model,trim,year,origin,color,options,fuel,gear,drivetrain,cylinders,engine size,condition,kilometer,location,aminities,cabin
0,Kia,Sportage,LX,2019,Saudi,White,Standard,Gasoline,Automatic,Double (4x4),4.0,2.0,Used,97263 km,Riyadh,"['Dual Airbag (Driver & Passenger)', 'ABS', 'C...",NaN
1,Toyota,Yaris,Y,2021,Saudi,Brown,Standard,Gasoline,CVT,FWD,4.0,1.5,Used,56333 km,Riyadh,"['Electronic Brakeforce Distribution', 'Brake ...",NaN
2,Toyota,Yaris,Y,2021,Saudi,another_color,Standard,Gasoline,CVT,FWD,4.0,1.5,Used,61452 km,Riyadh,"['Electronic Brakeforce Distribution', 'Brake ...",NaN
3,Kia,Sportage,LX,2019,Saudi,White,Standard,Gasoline,Normal,Double (4x4),4.0,2.0,Used,108213 km,Riyadh,"['Dual Airbag (Driver & Passenger)', 'ABS', 'C...",NaN
4,Kia,Sportage,LX,2019,Saudi,White,Standard,Gasoline,Automatic,Double (4x4),4.0,2.0,Used,100642 km,Riyadh,"['Dual Airbag (Driver & Passenger)', 'ABS', 'C...",NaN


In [86]:
df_all.shape

(1187, 17)

In [87]:
df.columns

Index(['brand', 'model', 'trim', 'year', 'origin', 'color', 'options', 'fuel',
       'gear', 'drivetrain', 'cylinders', 'engine size', 'condition',
       'kilometer', 'location', 'aminities', 'cabin'],
      dtype='object')

In [32]:
#df.to_csv('../Data/car_switch_data.csv',index=False)

In [89]:
df_all_carswitch=pd.read_csv('../Data/car_switch_data.csv')

In [90]:
df_all_carswitch.head()

,title,price,model_year,odo,aminities,specs,number_of_seats,drive_type,number_of_cylinders,body_type,...,first_owner,accidents_(not_verified),currently_financed?,seller_nationality,no._of_keys,roof_type,color,service_history,warranty,service_contract
0,Nissan X-Trail I4 2.5L,"\nSAR 54,000",2016,"72,000 KM","['Cruise Control', 'dynamic radar cruise contr...",GCC /Saudi Specs,5.0,2WD,4,SUV,...,Yes,No,No,India,2.0,NaN,NaN,NaN,NaN,NaN
1,Chrysler 300 V6 3.6L Limited,"\nSAR 30,000",2013,"207,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,5.0,2WD,6,Sedan,...,NaN,3 or more accidents,No,Saudi Arabia,1.0,NaN,NaN,NaN,NaN,NaN
2,Nissan Altima I4 2.5L S,"\nSAR 20,500",2010,"273,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,5.0,2WD,4,Sedan,...,NaN,No,No,Yemen,1.0,Sun Roof,NaN,NaN,NaN,NaN
3,GMC Acadia V6 3.6L,"\nSAR 22,000",2012,"135,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,7.0,2WD,6,SUV,...,NaN,No,No,Egypt,2.0,NaN,NaN,NaN,NaN,NaN
4,Mazda 6 I4 2.5L,"\nSAR 55,000",2017,"87,000 KM",['front & rear park sensors front parking sens...,GCC /Saudi Specs,5.0,2WD,4,Sedan,...,Yes,No,No,Pakistan,1.0,NaN,Red,Yes in Garage,NaN,NaN


In [91]:
df_all_carswitch.columns

Index(['title', 'price', 'model_year', 'odo', 'aminities', 'specs',
       'number_of_seats', 'drive_type', 'number_of_cylinders', 'body_type',
       'transmission_type', 'fuel_type', 'rim_size', 'first_owner',
       'accidents_(not_verified)', 'currently_financed?', 'seller_nationality',
       'no._of_keys', 'roof_type', 'color', 'service_history', 'warranty',
       'service_contract'],
      dtype='object')

In [13]:
df_all.shape

(3340, 23)

In [17]:
# check for duplication
df_all[df_all.duplicated()].shape

(47, 23)

In [93]:
df_all_carswitch['transmission_type']

0       Automatic
1       Automatic
2       Automatic
3       Automatic
4       Automatic
          ...    
3335    Automatic
3336    Automatic
3337    Automatic
3338    Automatic
3339    Automatic
Name: transmission_type, Length: 3340, dtype: object